## Instalação de Dependências

In [ ]:
!pip install numpy
!pip install pandas
!pip install folium

## Lendo os dados do arquivo Json

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium.plugins import HeatMap
import random
from datetime import datetime, timedelta

#df_traffic = pd.read_csv('../input/ukTrafficAADF.csv')
df = pd.read_json('./data.json', dtype=object)
df['time_utc'] = pd.to_datetime(df['time_utc'].str.split(' +', expand=True)[0])
df.head()

,MMSI,MMSI_String,ShipName,latitude,longitude,time_utc
0,357787000,357787000,UNI PATRIOT,10.757568,106.79384,2023-08-30
1,525100633,525100633,SEIRA,-5.932788,105.976182,2023-08-30
2,563080370,563080370,PENGUIN REFLECTION,1.223018,103.774287,2023-08-30
3,574005410,574005410,PRIME,20.84365,106.768217,2023-08-30
4,563336000,563336000,HY JADE,1.302853,103.950658,2023-08-30


## Mapa de Calor simples

In [13]:
# Define a data de início e fim para o período de 7 dias
end_date = df['time_utc'].max()
start_date = end_date - timedelta(days=7)

# Filtra os dados para o período de 7 dias
filtered_df = df[(df['time_utc'] >= start_date) & (df['time_utc'] <= end_date)]

# Agrupa os dados por latitude e longitude e contar a frequência
heat_data = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='frequency')

# Cria um mapa
map_hooray = folium.Map(location=[heat_data['latitude'].mean(), heat_data['longitude'].mean()], zoom_start=5)

# Cria o mapa de calor com base na frequência
heat_map = HeatMap(data=heat_data[['latitude', 'longitude', 'frequency']],
                   radius=15, blur=20, max_zoom=10)

# Adiciona o mapa de calor ao mapa
heat_map.add_to(map_hooray)

# Exibe o mapa
map_hooray

## Mapa de calor com densidade

* Densidade - Quantidade de posições de navios que cruzaram em um grid
* Grid - Tamanho em bloco de uma área no mapa

In [14]:
# Define a data de início e fim para o período de 7 dias
end_date = df['time_utc'].max()
start_date = end_date - timedelta(days=7)

# Filtra os dados para o período de 7 dias
df = df[(df['time_utc'] >= start_date) & (df['time_utc'] <= end_date)]

# Pega a latitude e longitude média entre os pontos
center_lat, center_lon = df['latitude'].mean(), df['longitude'].mean()

# Utiliza a lat e long média para gerar um mapa com um zoom centralizado na média dos pontos
map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Define o tamanho dos grids em graus - medida para mapa plano
grid_size = 0.1

# Cria os grids com base nas latitudes e longitudes
df['grid_lat'] = (df['latitude'] // grid_size) * grid_size
df['grid_lon'] = (df['longitude'] // grid_size) * grid_size

# Agrupa os dados por grids e calcula a densidade das embarcações
density_data = df.groupby(['grid_lat', 'grid_lon']).size().reset_index(name='density')

# Converte os dados em uma lista de coordenadas junto com a densidade para o HeatMap
heat_data = [[row['grid_lat'], row['grid_lon'], row['density']] for _, row in density_data.iterrows()]

# Adiciona os dados ao HeatMap
HeatMap(heat_data).add_to(map)

# Exibe o mapa
map